In [10]:
import sys
from pathlib import Path

# Add project root (one level above 'src') to PYTHONPATH
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.append(str(project_root))

In [11]:
import pandas as pd
from pathlib import Path
from src.optimizer import mean_variance_optimizer
from src.backtest import backtest
from src.metric import all_metrics, rmse, directional_accuracy, paired_ttest, jobson_korkie_memmel


## 1️⃣ Load predictions & realized returns


In [12]:
from pathlib import Path

project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()

preds_path = project_root / 'results' / 'predictions_transformer.csv'
preds = pd.read_csv(preds_path, index_col=0, parse_dates=True)

targets_path = project_root / 'data' / 'processed' / 'targets.csv'
realized = pd.read_csv(targets_path, index_col=0, parse_dates=True)

# Align dates & assets
common_idx = preds.index.intersection(realized.index)
preds = preds.loc[common_idx]
realized = realized.loc[common_idx, preds.columns]

## 2️⃣ Portfolio optimization

In [13]:
weights = mean_variance_optimizer(preds, realized)

/home/khushdeep/miniconda3/envs/project/lib/python3.11/site-packages/pandas/core/frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/home/khushdeep/miniconda3/envs/project/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/home/khushdeep/miniconda3/envs/project/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


## 3️⃣ Backtest portfolio


In [14]:
port_rets = backtest(weights, realized)

## 4️⃣ Compute metrics

In [15]:
perf_metrics = all_metrics(port_rets, weights)
print("Portfolio Performance Metrics:")
for k, v in perf_metrics.items():
    print(f"{k}: {v:.4f}")

Portfolio Performance Metrics:
ann_return: -1.1599
ann_vol: 1.0332
sharpe: -1.1226
max_dd: -1.0185
calmar: -1.1388
sortino: -0.8419
turnover: 4.6543


## 5️⃣ Forecast accuracy

In [16]:

forecast_rmse = rmse(realized, preds).mean()
forecast_dir_acc = directional_accuracy(realized, preds).mean()

print(f"Forecast RMSE: {forecast_rmse:.6f}")
print(f"Directional Accuracy: {forecast_dir_acc:.2%}")


Forecast RMSE: 0.013680
Directional Accuracy: 48.96%


## 6️⃣ Statistical tests vs benchmark


In [17]:
preds_path = project_root / 'results' / 'metrics_lstm.csv'
# Example: benchmark = equally-weighted returns
eq_weights = pd.DataFrame(1/len(preds.columns), index=preds.index, columns=preds.columns)
benchmark_rets = backtest(eq_weights, realized)

t_stat, p_val = paired_ttest(port_rets, benchmark_rets)
print(f"Paired t-test: t={t_stat:.4f}, p={p_val:.4f}")

z_stat, p_val_z = jobson_korkie_memmel(port_rets, benchmark_rets)
print(f"Jobson-Korkie-Memmel: z={z_stat:.4f}, p={p_val_z:.4f}")

# Save metrics to CSV
pd.DataFrame([perf_metrics]).to_csv(preds_path, index=False)

Paired t-test: t=-2.0682, p=0.0389
Jobson-Korkie-Memmel: z=-697.3012, p=0.0000


In [18]:
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np

fig_path = project_root / 'results' / 'figures'
fig_dir = Path(fig_path)
fig_dir.mkdir(parents=True, exist_ok=True)

# Cumulative returns plot
cum_rets = (1 + port_rets).cumprod()
cum_bench = (1 + benchmark_rets).cumprod()

plt.figure(figsize=(10,6))
plt.plot(cum_rets, label="Model Portfolio")
plt.plot(cum_bench, label="Equal Weight Benchmark")
plt.title("Cumulative Returns")
plt.xlabel("Date")
plt.ylabel("Growth of $1")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(fig_dir / "cumulative_returns.png", dpi=300)
plt.close()

# Rolling Sharpe ratio
rolling_sharpe = (port_rets.rolling(60).mean() / port_rets.rolling(60).std()) * np.sqrt(252)
plt.figure(figsize=(10,6))
plt.plot(rolling_sharpe, label="Rolling Sharpe (60d)")
plt.title("Rolling Sharpe Ratio")
plt.xlabel("Date")
plt.ylabel("Sharpe Ratio")
plt.grid(True)
plt.tight_layout()
plt.savefig(fig_dir / "rolling_sharpe.png", dpi=300)
plt.close()

# Heatmap of portfolio weights
plt.figure(figsize=(12,6))
sns.heatmap(weights.T, cmap="viridis", cbar=True)
plt.title("Portfolio Weights Over Time")
plt.xlabel("Date")
plt.ylabel("Asset")
plt.tight_layout()
plt.savefig(fig_dir / "weights_heatmap.png", dpi=300)
plt.close()

print(f"Figures saved in {fig_dir}")


Figures saved in /home/khushdeep/Desktop/TODO/universa analytics/results/figures
